## Initialization

In [1]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/project/

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/project


In [ ]:
!pip install transformers==4.28.0
!pip install datasets

In [ ]:
!pip install evaluate

## Actual Training

In [ ]:
import datasets
from datasets import load_dataset

dataset = load_dataset("csv",data_files={'train': 'fedspeak_data_train_medium_v2_train.csv', "validation":'fedspeak_data_train_medium_v2_val.csv', 'test': 'fedspeak_data_test_medium.csv'})
dataset

In [ ]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", fast=False)
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["Sentence"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Sentence"])

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [ ]:
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8)

In [ ]:
from transformers import BertForSequenceClassification
#checkpoint = "microsoft/deberta-base"
checkpoint = "./FinBERT-Prime_128MSL-500K"
#checkpoint = "roberta-base"
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

## FinBERT Training

In [9]:
import numpy as np
import evaluate
from datasets import load_metric
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import TrainingArguments, Trainer
import torch
training_args = TrainingArguments(per_device_train_batch_size = 5, per_device_eval_batch_size=5, output_dir="FinBERT-Prime_128MSL-500K_1",
                                  evaluation_strategy="epoch", num_train_epochs=20, seed=3407, optim="adamw_torch",
                                  learning_rate = 0.00001, weight_decay = 0.001,
                                  load_best_model_at_end=False, save_strategy='no',
                                  metric_for_best_model='accuracy')
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.986556,0.709302
2,No log,3.133118,0.732558
3,No log,3.202475,0.744186
4,No log,3.276132,0.744186
5,No log,3.560113,0.720930
6,No log,3.491108,0.709302
7,No log,3.503007,0.732558
8,0.000000,3.525422,0.732558
9,0.000000,3.545042,0.732558
10,0.000000,3.562411,0.732558


TrainOutput(global_step=1380, training_loss=1.8566186106223883e-06, metrics={'train_runtime': 694.7574, 'train_samples_per_second': 9.874, 'train_steps_per_second': 1.986, 'total_flos': 1804990457303040.0, 'train_loss': 1.8566186106223883e-06, 'epoch': 20.0})

In [ ]:
tokenized_small_datasets = small_dataset.map(tokenize_function, batched=True)

In [19]:
tokenized_small_datasets = tokenized_small_datasets.remove_columns(["Sentence"])

In [ ]:
small_dataset = load_dataset("csv",data_files={'test': 'fedspeak_data_test.csv'})

In [20]:
predictions = trainer.predict(tokenized_small_datasets['test'])
true_labels = tokenized_small_datasets['test']["labels"]
pred_labels = list(predictions.predictions.argmax(-1))
print(f"Accuracy ({data_selected}): {predictions.metrics['test_accuracy']}")
print(f"True_labels (n={len(true_labels)}): {true_labels}")
print(f"Pred_labels (n={len(true_labels)}): {pred_labels}\n\n")

Accuracy (['train', 'validation', 'test']): 0.5714285714285714
True_labels (n=21): [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4]
Pred_labels (n=21): [0, 0, 3, 0, 2, 2, 0, 3, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 4, 3, 0]


